In [1]:
import re
from tqdm import tqdm

In [2]:
import openai
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from itertools import chain

In [3]:
import urllib3
import json

def nerByETRI(text):
    openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU" 
    accessKey = "26890e16-f465-40c0-8881-37074821de8a"
    analysisCode = "ner"
    text = text

    requestJson = {  
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }

    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8", "Authorization" :  accessKey},
        body=json.dumps(requestJson)
    )

    result = str(response.data,"utf-8")
    json_dict = json.loads(result)
    ner = json_dict['return_object']['sentence'][0]['NE'][0]['type']
    return ner

In [4]:
from_date = '20020101'
to_date = '20230425'
topic = '재배 AND 작물'

cwd = os.getcwd()
path = cwd +'/' + topic + '/raw'
file_list = os.listdir(path)

dfs = []
for i in file_list:
    df = pd.read_excel(path+'/'+i)
    dfs.append(df)
    print(i)
data = pd.concat(dfs)

C:\Users\NH\anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20020101-20091231.xlsx
NewsResult_20100101-20121231.xlsx
NewsResult_20130101-20161231.xlsx
NewsResult_20170101-20201231.xlsx
NewsResult_20210101-20230425.xlsx


In [5]:
# 데이터 전처리

In [6]:
data = data.sort_values(by = '일자')

In [7]:
df = data[['일자','키워드']]
df.reset_index(inplace = True, drop = True)

In [8]:
# 1 행정구역
import geopandas as gpd

In [9]:
# prepare and read data
F1 = cwd +'/' + topic  + '/CTPRVN_202302/ctp_rvn.shp' 
D1 = gpd.read_file(F1, encoding='cp949')

F2 = cwd +'/' + topic  + '/SIG_202302/sig.shp' 
D2 = gpd.read_file(F2, encoding='cp949')

D1 = D1[['CTPRVN_CD','CTP_KOR_NM']]
D2 = D2[['SIG_CD','SIG_KOR_NM']]

In [10]:
# 시군구 달기
D2['SIG_KOR_NM_SINGLE'] = [x[:-1] for x in D2['SIG_KOR_NM']]
tmp = []
for i in D2['SIG_KOR_NM_SINGLE']:
    if len(i) > 3:
        i = i[0:2]
        tmp.append(i)
    else:
        tmp.append(i)
D2['SIG_KOR_NM_SINGLE'] = tmp

regions = []

for j in tqdm(df['키워드']):
    for i in j.split(','):
        check = 0
        if i in list(D2['SIG_KOR_NM']) :
            check = 1
            regions.append(i)
            break
        elif i in list(D2['SIG_KOR_NM_SINGLE']) :
            check = 1
            regions.append(i)
            break
    if check == 0:
        regions.append('nan')

100%|███████████████████████████████████████████████████████████████████████████| 83628/83628 [02:23<00:00, 583.26it/s]


In [11]:
D1

,CTPRVN_CD,CTP_KOR_NM
0,11,서울특별시
1,26,부산광역시
2,27,대구광역시
3,28,인천광역시
4,29,광주광역시
5,30,대전광역시
6,31,울산광역시
7,36,세종특별자치시
8,41,경기도
9,42,강원도


In [12]:
# 시도 달기
D1['CTP_KOR_NM_SINGLE'] = [x[0:2] for x in D1['CTP_KOR_NM']]
D1['CTP_KOR_NM_SINGLE'][10] = '충북'
D1['CTP_KOR_NM_SINGLE'][11] = '충남'
D1['CTP_KOR_NM_SINGLE'][12] = '전북'
D1['CTP_KOR_NM_SINGLE'][13] = '전남'
D1['CTP_KOR_NM_SINGLE'][14] = '경북'
D1['CTP_KOR_NM_SINGLE'][14] = '경남'

provines = []

for j in tqdm(df['키워드']):
    for i in j.split(','):
        check = 0
        if i in list(D1['CTP_KOR_NM_SINGLE']) :
            check = 1
            provines.append(i)
            break
    if check == 0:
        provines.append('nan')

100%|██████████████████████████████████████████████████████████████████████████| 83628/83628 [00:45<00:00, 1854.47it/s]


In [13]:
# 병합
df['시도'] = provines
df['시군구'] = regions

C:\Users\NH\AppData\Local\Temp\ipykernel_9532\581372460.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['시도'] = provines
C:\Users\NH\AppData\Local\Temp\ipykernel_9532\581372460.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['시군구'] = regions


In [15]:
# 시군구 코드 달기
codes = []
new_regions = []
for i in df['시군구']:
    try:
        code = D2[(D2['SIG_KOR_NM_SINGLE'] == i) | (D2['SIG_KOR_NM'] == i)]['SIG_CD'].values[0]
        region = D2[(D2['SIG_KOR_NM_SINGLE'] == i) | (D2['SIG_KOR_NM'] == i)]['SIG_KOR_NM'].values[0]
        codes.append(code)
        new_regions.append(region)
    except:
        codes.append('nan')
        new_regions.append('nan')

In [16]:
df['code'] = codes
# 용어 통일
df['시군구'] = new_regions

C:\Users\NH\AppData\Local\Temp\ipykernel_9532\1487141408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['code'] = codes
C:\Users\NH\AppData\Local\Temp\ipykernel_9532\1487141408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['시군구'] = new_regions


In [17]:
# 결측치 제거
df = df[(df['시도'] != 'nan') | (df['시군구'] != 'nan') | (df['code'] != 'nan')]
df.reset_index(drop = True, inplace= True)

In [18]:
# 시군구 코드에서 시도코드 파싱
df['code_provine'] = [x[0:2] for x in df['code']]

code_provine_dict = dict(zip(D1['CTPRVN_CD'], D1['CTP_KOR_NM']))

C:\Users\NH\AppData\Local\Temp\ipykernel_9532\2157452940.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['code_provine'] = [x[0:2] for x in df['code']]


In [19]:
provinesingle_provine = dict(zip(D1['CTP_KOR_NM_SINGLE'], D1['CTP_KOR_NM']))

In [20]:
for i,j in enumerate(df['시도']):
    if j != 'nan':
        df['시도'][i] = provinesingle_provine[j]

C:\Users\NH\AppData\Local\Temp\ipykernel_9532\3587699959.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['시도'][i] = provinesingle_provine[j]


In [21]:
provine_code_dict =  dict(zip(D1['CTP_KOR_NM'], D1['CTPRVN_CD']))

In [22]:
for i, j in enumerate(df['code_provine']):
    if j == 'na':
        df['code_provine'][i] = provine_code_dict[df['시도'][i]]

C:\Users\NH\AppData\Local\Temp\ipykernel_9532\2089381424.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['code_provine'][i] = provine_code_dict[df['시도'][i]]


In [23]:
# 시도코드로 시도추정
provines_est = []
for i in df['code_provine']:
    try:
        provines_est.append(code_provine_dict[i])
    except:
        provines_est.append('nan')
        
df['시도'] = provines_est

C:\Users\NH\AppData\Local\Temp\ipykernel_9532\1777164067.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['시도'] = provines_est


In [24]:
# 2. 재배작물을 NER -> 안되면 해서 추출... 안되면 사전 이용

In [25]:
# 키워드 카운트 하고 500번 이상 출현한 단어만 1차 선정해서 NER
# 우선 ETRI의 AP!의 1일 한도를 감안해 5000개 미만 NER, 
# ETRI가 농축산품 개체명 인식은 정확하지 않고 제한도 있어서 더 나은 개체명 인식 필요
df['키워드'] = [x.split(',') for x in df['키워드']]

kwd_cnt = Counter(list(chain(*df['키워드'].values)))
kwd_cnt_df = pd.DataFrame(kwd_cnt, index = ['value']).T
kwd_cnt_df = kwd_cnt_df[kwd_cnt_df['value'] > 500] 
kwd_cnt_df.index

C:\Users\NH\AppData\Local\Temp\ipykernel_9532\49188014.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['키워드'] = [x.split(',') for x in df['키워드']]


Index(['약속', '3년', '도시', '생활', '귀농', '채소', '비닐하우스', '실패', '주위', '겨울',
       ...
       'IoT', '더불어민주당', 'PLS', '청년농', '혁신밸리', '과수화상병', '코로나', '코로나19', '윤석열',
       '가루쌀'],
      dtype='object', length=3309)

In [26]:
# 한번만!
ners = []

for i in tqdm(kwd_cnt_df.index):
    try:
        ner = nerByETRI(i)
        ners.append(ner)
    except:
        ners.append('etc')

100%|██████████████████████████████████████████████████████████████████████████████| 3309/3309 [03:54<00:00, 14.10it/s]


In [27]:
# 대분류 PT만 사용
kwd_ner_df = pd.DataFrame({'kwd':kwd_cnt_df.index,'ner':ners})
kwd_ner_df = kwd_ner_df[kwd_ner_df['ner'] != 'etc']
idx = ['PT' in x for x in kwd_ner_df['ner'] ]
pt_df = kwd_ner_df[idx]
pt_df.reset_index(inplace=True, drop=True)
pt_list = pt_df['kwd'].values
pt_list = list(pt_list)

In [29]:
# 개체명 잘못된거 제거

remove_kwd = []

for i in remove_kwd:
    pt_list.remove(i)

In [30]:
# 개체명에서 누락된 것 추가


In [31]:
# 작물별, 연도별, 지역별 집적된 표를 만들어야함.

In [32]:
for i,_ in tqdm(enumerate(df['키워드'])):
    new_kwd = list((set(pt_list) & set(df['키워드'][i])))
    df['키워드'][i] = new_kwd

0it [00:00, ?it/s]C:\Users\NH\AppData\Local\Temp\ipykernel_9532\3018013306.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['키워드'][i] = new_kwd
70267it [00:58, 1202.07it/s]


In [133]:
idx = [len(x) != 0 for x in df['키워드']]
result_df = df[idx]

In [199]:
len(pt_list)

110

In [135]:
result_df = result_df.explode("키워드").reset_index(drop=True)

In [138]:
result_df['연도'] = [str(x)[0:4] for x in result_df['일자']]

In [139]:
result_df

,일자,키워드,시도,시군구,code,code_provine,연도
0,20020101,호박,경기도,양주시,41630,41,2002
1,20020101,키위,경기도,양주시,41630,41,2002
2,20020101,오이,경기도,양주시,41630,41,2002
3,20020101,작물,경기도,양주시,41630,41,2002
4,20020101,딸기,경기도,양주시,41630,41,2002
...,...,...,...,...,...,...,...
178334,20230425,배추,울산광역시,nan,nan,31,2023
178335,20230425,대파,울산광역시,nan,nan,31,2023
178336,20230425,작물,울산광역시,nan,nan,31,2023
178337,20230425,양파,울산광역시,nan,nan,31,2023


In [140]:
result_df = result_df.groupby(['연도','시도','키워드']).size().unstack(fill_value=0).stack().to_frame()

In [141]:
result_df = result_df.reset_index()
result_df = result_df.rename({0:'value'}, axis = 1)

In [183]:
result_df

,연도,시도,키워드,value
0,2002,강원도,가지,16
1,2002,강원도,감귤,5
2,2002,강원도,감자,31
3,2002,강원도,감초,1
4,2002,강원도,고사리,0
...,...,...,...,...
40365,2023,충청북도,한라봉,0
40366,2023,충청북도,헤어리베치,0
40367,2023,충청북도,호두,0
40368,2023,충청북도,호밀,2


In [168]:
def resultNormalize(result_df, pt, region):
    result_df_agg = pd.DataFrame(result_df.groupby(['연도','키워드']).sum('value').unstack(fill_value=0).stack()).reset_index()
    region_df = result_df[result_df['시도'] == region][result_df['키워드'] == pt]
    normalize_df = region_df.copy()
    
    idxs = []
    for tmp in result_df_agg[result_df_agg['키워드'] == pt]['연도']:
        idx = tmp in list(normalize_df['연도'])
        idxs.append(idx)

    normalize_df['value'] = region_df['value'].values / result_df_agg[result_df_agg['키워드'] == pt][idxs]['value'].values
    return normalize_df



In [144]:
from scipy import stats

def getslop(normalize_df):
    x = [x + 1for x in range(0,len(normalize_df))]
    y = normalize_df['value'].values
    slope, intercept, r, p, std_err = stats.linregress(x, y)
    return slope    

In [221]:
pt = '복숭아'
region = '강원도'

In [222]:
result_df_agg = pd.DataFrame(result_df.groupby(['연도','키워드']).sum('value').unstack(fill_value=0).stack()).reset_index()
region_df = result_df[result_df['시도'] == region][result_df['키워드'] == pt]

C:\Users\NH\AppData\Local\Temp\ipykernel_9532\4152275024.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  region_df = result_df[result_df['시도'] == region][result_df['키워드'] == pt]


In [223]:
region_df 

,연도,시도,키워드,value
46,2002,강원도,복숭아,8
1806,2003,강원도,복숭아,1
3566,2004,강원도,복숭아,5
5326,2005,강원도,복숭아,4
7086,2006,강원도,복숭아,3
8846,2007,강원도,복숭아,6
10606,2008,강원도,복숭아,24
12476,2009,강원도,복숭아,24
14236,2010,강원도,복숭아,53
16106,2011,강원도,복숭아,11


In [42]:
# 지도 시각화!

In [176]:
idxs = []
regions = list(set(result_df['시도']))

In [177]:
slopes = []
for region in tqdm(regions):
    for pt in pt_list:
        normalize_df = resultNormalize(result_df,pt,region)
        slope = getslop(normalize_df) 
        idx = region+ ' ' + pt
        
        slopes.append(slope)
        idxs.append(idx)

  0%|                                                                                           | 0/17 [00:00<?, ?it/s]C:\Users\NH\AppData\Local\Temp\ipykernel_9532\1255731921.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  region_df = result_df[result_df['시도'] == region][result_df['키워드'] == pt]
C:\Users\NH\AppData\Local\Temp\ipykernel_9532\1255731921.py:11: RuntimeWarning: invalid value encountered in divide
  normalize_df['value'] = region_df['value'].values / result_df_agg[result_df_agg['키워드'] == pt][idxs]['value'].values
100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:24<00:00,  1.44s/it]


In [182]:
pd.DataFrame({'region_pt':idxs,'slope':slopes}).sort_values(by = 'slope', ascending=False)[:50]

,region_pt,slope
1165,전라북도 복분자,0.024063
1840,경상북도 방풍,0.020987
1461,충청남도 표고,0.017631
752,경상남도 코스모스,0.017204
844,전라남도 산수유,0.014782
562,충청북도 복숭아,0.014659
1507,충청남도 쪽파,0.014620
1830,경상북도 한라봉,0.014063
1510,충청남도 방풍,0.014049
512,인천광역시 선인장,0.013721
